He creado `core/kelly_analysis.py` con las siguientes funciones clave:

1. `simular_trayectorias_monte_carlo(...)`
   → genera curvas de equity aleatorias usando Kelly.

2. `calcular_expected_shortfall(...)`
   → calcula el Expected Shortfall (CVaR) al nivel que indiques (por defecto 5 %).

3. `test_validacion_binomial(...)`
   → realiza un test binomial clásico para validar si el win-rate observado es coherente con el teórico.

4. `resumen_simulacion(...)`
   → devuelve percentiles y estadísticas descriptivas sobre los resultados de las trayectorias.

In [24]:
# setup.py (o bien en la primera celda de cada notebook)
import sys
from pathlib import Path

# Ajusta según la ubicación del notebook respecto a la carpeta del proyecto
project_root = Path("..").resolve()  # sube un nivel si estás en /notebooks
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
    

# Importa las funciones necesarias desde el módulo core
from core import process_new_files, daily_summary_from_hist   
# 1️⃣  Primero cargas tu DataFrame y lo dejas en el namespace global
df_raw = process_new_files( # te pedirá el capital inicial en ventana emergente
    merge_fragments=False, 
    verbose=True)

# QA : Quality Assurance
from core.qa import check_integrity
check_integrity(df_raw)

df = merge_split_trades(df_raw)


# Métricas globales rápidas
df_summary = daily_summary_from_hist(df)

total_fragmented = df["components"].apply(len).gt(1).sum() if "components" in df.columns else 0

print("\nTrades totales :", len(df))
print(f"Número de trades fragmentados: {total_fragmented}")
print("Win-rate       :", (df['PnL_net'] > 0).mean())
print("Profit factor  :", df.loc[df['PnL_net'] > 0, 'PnL_net'].sum() /
                          df.loc[df['PnL_net'] < 0, 'PnL_net'].abs().sum())
print("Drawdown máx % :", 1 - df['equity'].div(df['equity'].cummax()).min())
print("Total PnL_net   :", df['PnL_net'].sum())
print("Última equity   :", df['equity'].iloc[-1])


INFO: 📦 Total trades cargados: 62
INFO: 💾 Datos guardados en PostgreSQL (trades).
INFO: ✅ Procesados 62 trades en total.


✅ Todos los chequeos de integridad superados

Trades totales : 53
Número de trades fragmentados: 5
Win-rate       : 0.5283018867924528
Profit factor  : 1.37227478403949
Drawdown máx % : 0.2973262032085562
Total PnL_net   : 226.25
Última equity   : 645.25


In [ ]:
# 1️⃣  Empuja df al namespace global (como ya hacías)
import types, sys
mod = types.ModuleType("core.global_state")
mod.df = df
sys.modules["core.global_state"] = mod

# 2️⃣  Calcula P y R desde df con la nueva función
from core.kelly_analysis import (
    extraer_parametros_desde_df,
    simular_trayectorias_monte_carlo,
    calcular_expected_shortfall,
    resumen_simulacion,
    test_validacion_binomial
)

P, R, meta = extraer_parametros_desde_df(df)
print(f"P = {P:.4f}  R = {R:.2f}  meta = {meta}")

# 3️⃣  Decide la fracción de Kelly y lanza Monte Carlo
from core.kelly_simulator_v4 import calcular_kelly

kelly_puro = calcular_kelly(P, R)
f = 1.0                    # 100 % de Kelly
n_trades = 100
paths = 10_000
cap0 = df['equity'].iloc[-1]

eq = simular_trayectorias_monte_carlo(P, R, kelly_puro * f,
                                      cap0=cap0, n=n_trades, n_paths=paths)

print("Resumen:", resumen_simulacion(eq))
es, p5 = calcular_expected_shortfall(eq)
print(f"CVaR 5 %: {es:.2f} (p5 = {p5:.2f})")

# 4️⃣  Test binomial (opcional, si quieres validar el win-rate)
wins = meta['wins']
losses = meta['losses']
p_val = test_validacion_binomial(wins, losses, p_teorico=P)
print(f"P-valor binomial: {p_val:.4f}")



P = 0.5283  R = 1.23  meta = {'wins': 28, 'losses': 25, 'p_mean': np.float64(0.5283018867924528), 'r_mean': np.float64(1.2252453428924017)}\ 
Resumen: {'p05': 189.00561011145757, 'p25': 670.2534030572355, 'p50': 2376.8586765487275, 'p75': 6142.23756201655, 'p95': 29890.42052748302, 'mean': 7498.633586266826, 'std': 19963.608242240316, 'min': 4.238202445416743, 'max': 707852.6403924761}
CVaR 5 %: 102.78 (p5 = 189.01)
P-valor binomial: 1.0000
